In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
tf.reset_default_graph()

In [ ]:
(x_train, y_train), (x_test, y_test)= mnist.load_data()
# print("Train pics: {}, Train labels: {}, Test pics: {}, Test labels: {}".format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))

In [ ]:
class CNN:
    def __init__(self, image_height, image_width, channels, num_classes):
        self.input_layer = tf.placeholder(dtype=tf.float32, shape=[None, image_height, image_width, channels])
        
        conv_layer_1 = tf.layers.conv2d(self.input_layer, filters=32, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
        pooling_layer_1 = tf.layers.max_pooling2d(conv_layer_1, pool_size=[2, 2], strides=2)
        
        conv_layer_2 = tf.layers.conv2d(pooling_layer_1, filters=32, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
        pooling_layer_2 = tf.layers.max_pooling2d(conv_layer_2, pool_size=[2, 2], strides=2)
        
        flattened_pooling = tf.layers.flatten(pooling_layer_2)
        dense_layer = tf.layers.dense(flattened_pooling, 1024, activation=tf.nn.relu)
        
        dropout = tf.layers.dropout(dense_layer, rate=0.4, training=True)
        outputs = tf.layers.dense(dropout, num_classes) 
        
        self.choice = tf.argmax(outputs, axis=1)
        
        self.probability = tf.nn.softmax(outputs)
        self.labels = tf.placeholder(dtype=tf.float32, name="labels")
        self.accuracy, self.accuracy_op = tf.metrics.accuracy(self.labels, self.choice)
        one_hot_labels = tf.one_hot(indices=tf.cast(self.labels, dtype=tf.int32), depth=num_classes)
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=one_hot_labels, logits=outputs)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3)
        self.train_operation = optimizer.minimize(loss=self.loss, global_step=tf.train.get_global_step())
        
        
        


In [ ]:
steps = 10000
batch_size = 32
image_height = 28
image_width = 28
color_channels = 1
num_classes = 10

path = "./checkpoints/"

if os.path.exists(path):
    load_checkpoint = True
else:
    load_checkpoint = False

performance_graph = np.array([])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))    
dataset = dataset.shuffle(buffer_size=train_labels.shape[0])
dataset = dataset.batch(batch_size)
dataset = dataset.repeat()

dataset_iterator = dataset.make_initializable_iterator()
next_element = dataset_iterator.get_next()

cnn = CNN(image_height, image_width, color_channels, num_classes)

saver = tf.train.Saver(max_to_keep=2)

if not os.path.exists(path):
    os.makedir(path)

